In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")
np.random.seed(203)
from tqdm import tqdm
import datetime
from collections import Counter
import re

from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from keras.models import Model,Sequential
from keras.layers import *
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras import backend as K
from keras.utils import to_categorical
from keras.optimizers import RMSprop, Adam

Using TensorFlow backend.


In [2]:
data_train = pd.read_csv('/Users/s0c02nj/Desktop/Mahindra/train_5CLrC8b/train.csv')
#data_dict = pd.read_csv('/Users/s0c02nj/Desktop/train_5CLrC8b/Data_Dictionary.xlsx')
data_test = pd.read_csv('/Users/s0c02nj/Desktop/Mahindra/test.csv')
data_sub = pd.read_csv('/Users/s0c02nj/Desktop/Mahindra/sample_submission.csv')

In [3]:
data_train_new = data_train

In [4]:
#Train data separating the response
x_train = data_train_new.drop('amount_spent_per_room_night_scaled',axis=1)
y_def =  data_train_new['amount_spent_per_room_night_scaled']

In [5]:
#Test data and train data merged for pre-processing
x_comb = pd.concat([x_train,data_test],sort=False)

Missing value current
+ season_holidayed_code        149
+ state_code_residence        7024

Replacing Missing values with 'unknown' token for now

In [6]:
#x_comb = x_comb.replace(np.nan, 0.0, regex=True)
#x_comb.head()

In [7]:
x_comb['season_holidayed_code'].fillna(x_comb['season_holidayed_code'].mode()[0], inplace=True)
x_comb['state_code_residence'].fillna(x_comb['state_code_residence'].mode()[0], inplace=True)

Time Related features engineering

In [8]:
from datetime import timedelta, date
x_comb['booking_date'] =  pd.to_datetime(x_comb['booking_date'], format='%d/%m/%y')
x_comb['checkin_date'] =  pd.to_datetime(x_comb['checkin_date'], format= '%d/%m/%y')
x_comb['checkout_date'] = pd.to_datetime(x_comb['checkout_date'],format= '%d/%m/%y')

In [9]:
#Duration stay
x_comb['Duration_stay'] = x_comb['checkout_date'] - x_comb['checkin_date']
x_comb['Duration_stay'] = x_comb['Duration_stay'].apply(lambda x:x.days)

In [10]:
# #Difference from booking date and checkin date
# x_comb['diff1'] = x_comb['checkin_date'] - x_comb['booking_date']
# x_comb['diff1'] = x_comb['diff1'].apply(lambda x:x.days)

# #Difference from booking date and checkout date
# x_comb['diff2'] = x_comb['checkout_date'] - x_comb['booking_date']
# x_comb['diff2'] = x_comb['diff2'].apply(lambda x:x.days)
x_comb.shape

(488189, 24)

In [11]:
#Date-time year for all the time variables
cols =['booking_date','checkin_date','checkout_date']


for i in tqdm(cols):
    x_comb[i +' '+'month'] = x_comb[i].dt.month
    x_comb[i +' '+'week'] =  x_comb[i].dt.week
    x_comb[i +' '+'year'] =  x_comb[i].dt.year
    x_comb[i +' '+'day'] =   x_comb[i].dt.day
        

100%|██████████| 3/3 [00:00<00:00,  8.53it/s]


In [12]:
x_comb = x_comb.drop(['booking_date','checkin_date','checkout_date'],axis=1)

In [13]:
x_comb.columns

Index(['reservation_id', 'channel_code', 'main_product_code', 'numberofadults',
       'numberofchildren', 'persontravellingid', 'resort_region_code',
       'resort_type_code', 'room_type_booked_code', 'roomnights',
       'season_holidayed_code', 'state_code_residence', 'state_code_resort',
       'total_pax', 'member_age_buckets', 'booking_type_code', 'memberid',
       'cluster_code', 'reservationstatusid_code', 'resort_id',
       'Duration_stay', 'booking_date month', 'booking_date week',
       'booking_date year', 'booking_date day', 'checkin_date month',
       'checkin_date week', 'checkin_date year', 'checkin_date day',
       'checkout_date month', 'checkout_date week', 'checkout_date year',
       'checkout_date day'],
      dtype='object')

In [42]:
total_cols = ['channel_code', 'main_product_code', 'numberofadults',
       'numberofchildren',  'resort_region_code',
       'resort_type_code', 'room_type_booked_code', 'roomnights',
       'season_holidayed_code', 'state_code_residence', 'state_code_resort',
       'total_pax', 'member_age_buckets', 'booking_type_code',
       'cluster_code', 'reservationstatusid_code', 'resort_id',
       'Duration_stay', 'booking_date month', 'booking_date week',
       'booking_date year', 'booking_date day', 'checkin_date month',
       'checkin_date week', 'checkin_date year', 'checkin_date day',
       'checkout_date month', 'checkout_date week', 'checkout_date year',
       'checkout_date day']

In [43]:
cat_cols = ['channel_code', 'main_product_code',
        'resort_region_code',
       'resort_type_code', 'room_type_booked_code',
       'season_holidayed_code', 'state_code_residence', 'state_code_resort',
       'member_age_buckets', 'booking_type_code',
       'cluster_code', 'reservationstatusid_code', 'resort_id',
       'booking_date month', 'booking_date week',
       'booking_date year', 'checkin_date month',
       'checkin_date week', 'checkin_date year',
       'checkout_date month', 'checkout_date week', 'checkout_date year']

In [44]:
cont_cols = list(set(total_cols) - set(cat_cols))

In [45]:
cont_cols

['Duration_stay',
 'numberofadults',
 'numberofchildren',
 'booking_date day',
 'checkout_date day',
 'checkin_date day',
 'roomnights',
 'total_pax']

In [46]:
x_comb['roomnights'] = x_comb['roomnights'].replace(-45, 45)

In [47]:
for col in tqdm(cat_cols):
    le = LabelEncoder()
    x_comb[col] = le.fit_transform(x_comb[col])

100%|██████████| 22/22 [00:00<00:00, 42.82it/s]


In [49]:
#x_comb = x_comb.drop(['reservation_id','persontravellingid','memberid'],axis=1)
x_comb = x_comb.drop(['memberid'],axis=1)

In [50]:
train_x = x_comb[0:341424]
test_x =  x_comb[341424:]

In [51]:
train_cat = train_x[cat_cols]
test_cat = test_x[cat_cols]

train_cont = train_x[cont_cols]
test_cont = test_x[cont_cols]

In [52]:
def get_train_test_data_cat(df):
    
    data=[]
    for col in cat_cols:
        data.append(df[col])
    
    return data

In [53]:
x_train_cat = get_train_test_data_cat(train_x)
x_test_cat  = get_train_test_data_cat(test_x)

In [54]:
x_train_comb = x_train_cat+[train_cont]
x_test_comb = x_test_cat+ [test_cont]

In [55]:
y_train = y_def

In [61]:
def model_deep():
    
    layer_cat  = []
    input_cat =  []
    
    #Categorical_var
    for i,categoical_var in tqdm(enumerate((cat_cols))): 
        
        no_of_unique_cat  = x_comb[categoical_var].nunique()
        embedding_size = min(np.ceil((no_of_unique_cat)/2), 50 )
        embedding_size = int(embedding_size)
        
        #Defining the input-----> branch_id
        input_catg = Input(shape=(1,))
        layer_catg =  Embedding(no_of_unique_cat+1 ,embedding_size,input_length=1,trainable=True)(input_catg)
        layer_catg =  Flatten()(layer_catg)
        layer_cat.append(layer_catg)
        input_cat.append(input_catg)
    
    
    #continuous var
    input_cont = Input(shape=(8,))
    layer_cont = Dense(5, activation='relu')(input_cont)
        
    #Merging
    layer_comb =  layer_cat + [layer_cont]
    layer_comb = concatenate(layer_comb)
    
    layer_dense = Dense(512, activation='relu')(layer_comb)
    layer_dense = Dropout(0.5)(layer_dense)
    
    layer_dense = Dense(256, activation='relu')(layer_comb)
    layer_dense = Dropout(0.5)(layer_dense)
    
    layer_dense = Dense(128, activation='relu')(layer_comb)
    layer_dense = Dropout(0.5)(layer_dense)
    
    #Final output
    layer_output = Dense(1,activation='relu')(layer_dense)
    
    #Comb_inpus
    input_comb = input_cat + [input_cont]
    
    #Final model
    model = Model(inputs= input_comb ,outputs=layer_output)
    
    return model
        
    

In [62]:
model1 = model_deep()
model1.summary()

22it [00:00, 34.53it/s]

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_72 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
input_73 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
input_74 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
input_75 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
input_76 (

In [63]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [64]:
model1.compile(loss=root_mean_squared_error, 
                   optimizer='Adam',
                   metrics=['mse'])

In [ ]:
history = model1.fit(x_train_comb,y_train,
                         batch_size = 512, 
                         epochs = 100, 
                         validation_split=0.16, 
                         verbose = 1)


Train on 286796 samples, validate on 54628 samples
Epoch 1/100
286796/286796 [==============================] - 5s 18us/step - loss: 1.7659 - mean_squared_error: 3.8927 - val_loss: 1.1192 - val_mean_squared_error: 1.2556
Epoch 2/100
286796/286796 [==============================] - 4s 13us/step - loss: 1.3777 - mean_squared_error: 1.9006 - val_loss: 1.1085 - val_mean_squared_error: 1.2317
Epoch 3/100
286796/286796 [==============================] - 4s 13us/step - loss: 1.3335 - mean_squared_error: 1.7809 - val_loss: 1.0837 - val_mean_squared_error: 1.1777
Epoch 4/100
286796/286796 [==============================] - 4s 13us/step - loss: 1.2910 - mean_squared_error: 1.6691 - val_loss: 1.0774 - val_mean_squared_error: 1.1642
Epoch 5/100
286796/286796 [==============================] - 4s 13us/step - loss: 1.2598 - mean_squared_error: 1.5894 - val_loss: 1.0693 - val_mean_squared_error: 1.1470
Epoch 6/100
286796/286796 [==============================] - 4s 13us/step - loss: 1.2321 - mean_squ

In [36]:
pred = model1.predict(x_test_comb)

In [37]:
data_sub['amount_spent_per_room_night_scaled']=pred

In [38]:
data_sub.to_csv('/Users/s0c02nj/Desktop/Mahindra/Sub_cat7.csv',index=False)